In [1]:
import csv

## Function for Mean
def getMean(sampleList):
    sampleSize = len(sampleList)
    totalSumIncome = 0
    for row in sampleList:
        totalSumIncome = row + totalSumIncome
    mean = totalSumIncome / sampleSize
    return mean

## Function target sample Standard Deviation
def getSampleSD(sampleList):
    mean = getMean(sampleList)
    sumOfSquares = 0
    sampleSize = len(sampleList)
    for row in sampleList:
        deviationScore = row - mean
        sumOfSquares = deviationScore**2 + sumOfSquares
    
    variance = sumOfSquares / (sampleSize-1)
    sampleSD = variance**0.5
    return sampleSD

## Sample Standard Error
def getSampleStandardError(sampleList):
    sampleSize = len(sampleList)
    sampleSD = getSampleSD(sampleList)
    # standard error of the mean for the population
    SSE = sampleSD / sampleSize**0.5
    return SSE

In [2]:

##
def getCriticalT(alpha, sampleList, testType = 'two-tailed'):
    sampleSize = len(sampleList)
    df = sampleSize - 1
    if df > 120:
        df = 120
    criticalT = 0
    
    if testType == 'two-tailed':
        with open('data/t_table.csv', newline='') as tFile:
            tData = csv.reader(tFile)
            for row in tData:
                if int(row[0]) == df:
                    if float(row[2]) >= (float(1-(alpha/2))):
                        criticalT = float(row[1])
                        break
   
    elif testType == 'one-tailed positive':
        with open('data/t_table.csv', newline='') as tFile:
            tData = csv.reader(tFile)
            for row in tData:
                if int(row[0]) == df:
                    if float(row[2]) >= (float(1-(alpha))):
                        criticalT = float(row[1])
                        break

    elif testType == 'one-tailed negative':
        with open('data/t_table.csv', newline='') as tFile:
            tData = csv.reader(tFile)
            for row in tData:
                if int(row[0]) == df:
                    if float(row[2]) >= (float(alpha)):
                        criticalT = float(row[1])
                        break
    return criticalT

In [3]:
##
def getPFrom(t, df):
    p = None
    if df > 120:
        df = 120
    if t > 6:
        p = 1.0
    elif t < -6:
        p = 0.0
    elif t > -6 and t < 6:
        with open('data/t_table.csv', newline='') as tFile:
            tData = csv.reader(tFile)
            for row in tData:
                if float(row[1]) == round(float(t), 2):
                    p = float(row[2])
    return p

In [4]:
##
def oneSampleTTest(sampleList, popMean, alpha, testType = 'two-tailed'):
    sampleMean = getMean(sampleList)
    sampleSize = len(sampleList)
    SSE = getSampleStandardError(sampleList)
    df = sampleSize-1
    significant = False
    
    t = (sampleMean - popMean)/SSE
    p = getPFrom(t, df)
    critT = getCriticalT(alpha, sampleList, testType)
    
    if testType == 'two-tailed':
        if t < -critT or t > critT:
            significant = True
    elif testType == 'one-tailed positive':
        if t > critT:
            significant = True
    elif testType == 'one-tailed negative':
        if t < -critT:
            significant = True
                
    return significant, t, critT, p

In [5]:
import csv

generalPopIncomeData = []
blackFemaleIncomeData = []

with open('data/usa.csv', newline='') as myFile:
    data = csv.reader(myFile)
    for row in data:
        if row[6] != 'INCTOT':
            if int(row[6]) > 1:
                if int(row[6]) != 999999:
                    if int(row[1]) > 17:
                        generalPopIncomeData.append(int(row[6]))
                        if int(row[2]) == 2: ## Black Women to General Population
                            if int(row[0]) == 2:
                                blackFemaleIncomeData.append(int(row[6]))

In [6]:

import random

blackFemaleIncomeSample = random.sample(blackFemaleIncomeData, 30)
popMean = getMean(generalPopIncomeData)
# print('Population Mean: ', popMean)
sampleMean = getMean(blackFemaleIncomeSample)
# print('Sample Mean: ', sampleMean)

alpha = 0.05
testType = 'one-tailed negative'

sigBool, sampleT, criticalT, p = oneSampleTTest(
                                                blackFemaleIncomeSample,
                                                popMean, 
                                                alpha, 
                                                testType)

if sigBool:
    print('The one sample T test is significant at alpha =', alpha,
         ' where p=', p, 'and t=', round(sampleT, 2), 
         '.\nCrtical T:', criticalT,
         '\nSample Mean: ', sampleMean, '\nPopulation Mean: ', round(popMean, 2))
    
if not sigBool:
    print('The one sample T test is NOT significant at alpha =', alpha,
         ' where p=', p, 'and t=', round(sampleT, 2), 
         '.\nCritical T:', criticalT,
         '\nSample Mean: ', sampleMean, '\nPopulation Mean: ', round(popMean, 2))   
    

The one sample T test is significant at alpha = 0.05  where p= 0.01803 and t= -2.12 .
Crtical T: -1.69 
Sample Mean:  32824.666666666664 
Population Mean:  47496.59
